In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import mean_squared_error, r2_score
from utils import EarlyStopping
import utils
import importlib
importlib.reload(utils)

dataset_names = ['FD002', 'FD004']

concatname = ''
for i in dataset_names:
    concatname += (i[4])

sensors = ['s_2', 's_3', 's_4', 's_7', 's_8', 's_9', 's_11', 's_12', 's_13', 's_14', 's_15', 's_17', 's_20', 's_21']
alpha = 0.1
threshold = 125
window_size = 30
sensor_size = len(sensors)
forecast_size = 1
batch_size = 64
num_layers = 1
beta = 1
learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 데이터셋과 데이터 로더를 저장할 딕셔너리
train_domains_loader = {}
val_domains_loader = {}

for dataset_name in dataset_names:
    X_train, y_train, X_val, y_val, X_test, y_test = utils.get_data(dataset_name, sensors, window_size, alpha, threshold)
    
    # 데이터를 텐서로 변환하고 디바이스로 이동
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

    # 딕셔너리에 저장
    train_domains_loader[dataset_name] = {
        "X_train": X_train,
        "y_train": y_train,
    }
    
    val_domains_loader[dataset_name] = {
        "X_val": X_val,
        "y_val": y_val,
    }

def calculate_mean_and_variance_per_sample(X):
    means = np.mean(X, axis=1)  # axis=1, 각 샘플 별로 평균 계산
    vars = np.var(X, axis=1)  # axis=1, 각 샘플 별로 분산 계산
    return means, vars

def mix_style_per_sample(X1, y1, X2, y2, lambda_beta):
    min_len = min(len(X1), len(X2))
    
    # 각 샘플에 대한 평균과 분산 계산
    means1, vars1 = calculate_mean_and_variance_per_sample(X1)
    means2, vars2 = calculate_mean_and_variance_per_sample(X2)
    
    # 샘플링할 데이터의 인덱스 결정
    indices1 = np.random.permutation(len(X1))[:min_len]
    indices2 = np.random.permutation(len(X2))[:min_len]
    
    # 혼합 비율 샘플링
    lambda_vals = np.random.beta(lambda_beta, lambda_beta, size=min_len)
    
    # MixStyle 적용
    X_mixed = np.empty_like(X1[:min_len])
    for i in range(min_len):
        idx1 = indices1[i]
        idx2 = indices2[i]
        lambda_val = lambda_vals[i]
        X_mixed[i] = lambda_val * (X1[idx1] - means1[idx1]) / np.sqrt(vars1[idx1]) + \
                     (1 - lambda_val) * (X2[idx2] - means2[idx2]) / np.sqrt(vars2[idx2])
        X_mixed[i] = X_mixed[i] * np.sqrt(lambda_val * vars1[idx1] + (1 - lambda_val) * vars2[idx2]) + \
                     lambda_val * means1[idx1] + (1 - lambda_val) * means2[idx2]      
    
    # y의 혼합을 위해 브로드캐스팅 가능한 형태로 lambda_vals 조정
    lambda_vals_reshaped = lambda_vals[:, None]
    
    # y 혼합
    y_mixed = lambda_vals_reshaped * y1[indices1] + (1 - lambda_vals_reshaped) * y2[indices2]     
    
    return X_mixed, y_mixed

# 도메인1, 2 각각의 X_train과 y_train 혼합
lambda_beta = 0.2
train_X_mixed, train_y_mixed = mix_style_per_sample(train_domains_loader[dataset_names[0]]["X_train"], train_domains_loader[dataset_names[0]]["y_train"],
                             train_domains_loader[dataset_names[1]]["X_train"], train_domains_loader[dataset_names[1]]["y_train"],
                             lambda_beta)
val_X_mixed, val_y_mixed = mix_style_per_sample(val_domains_loader[dataset_names[0]]["X_val"], val_domains_loader[dataset_names[0]]["y_val"],
                             val_domains_loader[dataset_names[1]]["X_val"], val_domains_loader[dataset_names[1]]["y_val"],
                             lambda_beta)

# 혼합된 데이터를 텐서로 변환
train_X_mixed_tensor = torch.tensor(train_X_mixed, dtype=torch.float32)
train_y_mixed_tensor = torch.tensor(train_y_mixed, dtype=torch.float32)
val_X_mixed_tensor = torch.tensor(val_X_mixed, dtype=torch.float32)
val_y_mixed_tensor = torch.tensor(val_y_mixed, dtype=torch.float32)

# Define the CNN model
class CNNFeatureExtractor(nn.Module):
    def __init__(self, input_channels):
        super(CNNFeatureExtractor, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        return x

# Define the GRU model
class BiGRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, drop_prob=0.2):
        super(BiGRU, self).__init__()
        
        self.gru = nn.GRU(input_dim, hidden_dim, batch_first=True, dropout=drop_prob, bidirectional=True)
        self.fc = nn.Linear(2*hidden_dim, output_dim)

    def forward(self, x):
        gru_out, _ = self.gru(x)
        gru_out = gru_out[:, -1, :]
        output = self.fc(gru_out)
        return output

# Function to extract features using CNN
def extract_features(cnn_model, data):
    cnn_model.eval()
    with torch.no_grad():
        features = cnn_model(data.permute(0, 2, 1).to(device))
        features = features.permute(0, 2, 1)
    return features

# Function to create DataLoader
def create_dataloader(X, y, batch_size):
    dataset = TensorDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

# Create CNN model and move to device
cnn_model = CNNFeatureExtractor(input_channels=sensor_size).to(device)

# Extract features using CNN
train_X_features = extract_features(cnn_model, train_X_mixed_tensor)
val_X_features = extract_features(cnn_model, val_X_mixed_tensor)

# Create DataLoaders
mixed_train_loader = create_dataloader(train_X_features, train_y_mixed_tensor, batch_size)
mixed_val_loader = create_dataloader(val_X_features, val_y_mixed_tensor, batch_size)

# Create GRU model and move to device
hidden_dim = 32
output_dim = 1
rnn_model = BiGRU(input_dim=train_X_features.shape[2], hidden_dim=hidden_dim, output_dim=output_dim).to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(rnn_model.parameters(), lr=learning_rate)

# Training loop with early stopping
train_losses = []
val_losses = []
early_stopping = EarlyStopping(patience=35, verbose=True, delta=0)
epochs = 35

for epoch in range(epochs):
    rnn_model.train()
    total_train_loss = 0.0
    for batch_idx, (inputs, targets) in enumerate(mixed_train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = rnn_model(inputs)
        train_loss = criterion(outputs, targets)
        train_loss.backward()
        optimizer.step()
        
        total_train_loss += train_loss.item()

    avg_train_loss = total_train_loss / len(mixed_train_loader)
    train_losses.append(avg_train_loss)
    
    rnn_model.eval()
    total_val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in mixed_val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = rnn_model(inputs)
            val_loss = criterion(outputs, targets)
            total_val_loss += val_loss.item()

    avg_val_loss = total_val_loss / len(mixed_val_loader)
    val_losses.append(avg_val_loss)
    
    if (epoch + 1) % 5 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')

    early_stopping(avg_val_loss, rnn_model)
    if early_stopping.early_stop:
        print("Early stopping")
        break

save_path = f"MixStyle_{window_size}_{forecast_size}.pth"
torch.save(rnn_model.state_dict(), save_path)
print(f"Best model weights saved to {save_path}")


Validation loss decreased (inf --> 3089.562601).  Saving model ...
Validation loss decreased (3089.562601 --> 1752.533138).  Saving model ...
Validation loss decreased (1752.533138 --> 1515.931825).  Saving model ...
Validation loss decreased (1515.931825 --> 968.565213).  Saving model ...
Epoch [5/35], Train Loss: 606.9663, Val Loss: 474.2458
Validation loss decreased (968.565213 --> 474.245758).  Saving model ...
Validation loss decreased (474.245758 --> 342.747177).  Saving model ...
Validation loss decreased (342.747177 --> 306.394882).  Saving model ...
Validation loss decreased (306.394882 --> 275.240949).  Saving model ...
Validation loss decreased (275.240949 --> 267.993475).  Saving model ...
Epoch [10/35], Train Loss: 211.3746, Val Loss: 259.1301
Validation loss decreased (267.993475 --> 259.130126).  Saving model ...
Validation loss decreased (259.130126 --> 251.435049).  Saving model ...
Validation loss decreased (251.435049 --> 249.243272).  Saving model ...
Validation los

In [2]:
utils.evaluateDataset_1DCNN("FD001", rnn_model)
utils.evaluateDataset_1DCNN("FD002", rnn_model)
utils.evaluateDataset_1DCNN("FD003", rnn_model)
utils.evaluateDataset_1DCNN("FD004", rnn_model)

FD001 RMSE 57.56209945678711 R2 -1.0633058547973633 score [993221.44]
FD002 RMSE 53.70090866088867 R2 -0.5640617609024048 score [535361.9]
FD003 RMSE 36.491580963134766 R2 0.13190943002700806 score [38099.766]
FD004 RMSE 53.88380432128906 R2 -0.5716667175292969 score [683330.5]


In [3]:
dataset = "FD001"

# sensors to work with: 14 sensors
#sensors = ['s_3', 's_4', 's_7', 's_11', 's_12']
sensors = ['s_2','s_3','s_4','s_7','s_8','s_9','s_11','s_12','s_13','s_14','s_15','s_17','s_20','s_21']
window_size = 30
alpha = 0.1
threshold = 125
X_train, y_train, X_val, y_val, X_test, y_test = utils.get_data(dataset, sensors, window_size, alpha, threshold)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).to(device)

    
train_X_features = extract_features(cnn_model, X_train_tensor)
# Create DataLoaders
mixed_train_loader = create_dataloader(train_X_features, y_train_tensor, batch_size)

# Ensure the model is in evaluation mode and move it to the correct device
rnn_model.eval()
rnn_model.to(device)

# Use the model to predict y_test
with torch.no_grad():
    #model.load_state_dict(torch.load('C:/Users/User/exploring-nasas-turbofan-dataset/weights/linear_model_weights_20_5.pth'))
    predictions_tensor = rnn_model(train_X_features)
    
    # Clamp the predictions to be within [0, 125]
    predictions_tensor = torch.clamp(predictions_tensor, 0, 125)
    
    predictions = predictions_tensor.cpu().numpy()

# Convert y_test_tensor to numpy array
y_actual_test = y_test_tensor.cpu().numpy()

def evaluate(y_true, y_hat, label='test'):
    mse = mean_squared_error(y_true, y_hat)
    rmse = np.sqrt(mse)
    variance = r2_score(y_true, y_hat)
    
    score = 0
    #y_true = y_true.cpu()
    #y_hat = y_hat.cpu()
    for i in range(len(y_hat)):
        if y_true[i] <= y_hat[i]:
            score = score + np.exp(-(y_true[i] - y_hat[i]) / 10.0) - 1
        else:
            score = score + np.exp((y_true[i] - y_hat[i]) / 13.0) - 1
    
    print('{} RMSE {} R2 {} score {}'.format(dataset, rmse, variance, score))

# Evaluate the model's performance on the test set
evaluate(y_actual_test, predictions, label='test')


ValueError: Found input variables with inconsistent numbers of samples: [100, 43523]

In [5]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import mean_squared_error, r2_score
from utils import EarlyStopping
import utils
import importlib
importlib.reload(utils)

# Dataset and sensor settings
dataset = "FD001"
sensors = ['s_2','s_3','s_4','s_7','s_8','s_9','s_11','s_12','s_13','s_14','s_15','s_17','s_20','s_21']
window_size = 30
alpha = 0.1
threshold = 125

# Load data
X_train, y_train, X_val, y_val, X_test, y_test = utils.get_data(dataset, sensors, window_size, alpha, threshold)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convert data to tensors and move to device
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).to(device)

# Ensure the model is in evaluation mode and move it to the correct device
cnn_model.eval()
rnn_model.eval()
cnn_model.to(device)
rnn_model.to(device)

# Extract features using CNN for test set
test_X_features = extract_features(cnn_model, X_test_tensor)

# Use the model to predict y_test
with torch.no_grad():
    predictions_tensor = rnn_model(test_X_features)
    
    # Clamp the predictions to be within [0, 125]
    predictions_tensor = torch.clamp(predictions_tensor, 0, 125)
    
    predictions = predictions_tensor.cpu().numpy()

# Convert y_test_tensor to numpy array
y_actual_test = y_test_tensor.cpu().numpy()

def evaluate(y_true, y_hat, label='test'):
    mse = mean_squared_error(y_true, y_hat)
    rmse = np.sqrt(mse)
    variance = r2_score(y_true, y_hat)
    
    score = 0
    for i in range(len(y_hat)):
        if y_true[i] <= y_hat[i]:
            score = score + np.exp(-(y_true[i] - y_hat[i]) / 10.0) - 1
        else:
            score = score + np.exp((y_true[i] - y_hat[i]) / 13.0) - 1
    
    print('{} RMSE {} R2 {} score {}'.format(dataset, rmse, variance, score))

# Evaluate the model's performance on the test set
evaluate(y_actual_test, predictions, label='test')


FD001 RMSE 13.223660469055176 R2 0.8911085724830627 score [259.93713]
